In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset,DataLoader
import torchvision
import pandas as pd

In [ ]:
#image is 28*28 i.e. 784 pixels and there is a label corresponding to each image
#although there are 26 characters but we use only 24. only 24 labels are there. j and z dont have labels here bcoz they cant be displayed with static images. we need videos.
#there are 785 columns. 1st column is for label, rest 784 are for pixels

from google.colab import drive
drive.mount('/content/drive')

#train_df=pd.read_csv('/content/drive/MyDrive/sign_mnist_train.csv')
#test_df=pd.read_csv('/content/drive/MyDrive/sign_mnist_valid.csv')

train_df=pd.read_csv('/content/sign_mnist_train.csv')
test_df=pd.read_csv('/content/sign_mnist_valid.csv')

Mounted at /content/drive


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#in Pytorch, we need to create these 3 functions for all the external datasets to make it as torch dataset

class MyDataset(Dataset):
  def __init__(self,df): # self is an object. df is the dataframe
    x_train=df.iloc[:,1:].values.reshape(-1,1,28,28)/255.0  # all the pixel values. preparing image data for the CNN Model.
    #iloc[:,1:]=all rows and columns starting from 2nd
    y_train=df.iloc[:,0].values  #labels
    self.xs=torch.tensor(x_train,dtype=torch.float32) #converting to tensors
    self.ys=torch.tensor(y_train,dtype=torch.long)
  def __getitem__(self, index):
    return self.xs[index],self.ys[index]  #returning the index
  def __len__(self):
    return len(self.xs)


In [ ]:
train_df.head()

,label,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
0,3,107,118,127,134,139,143,146,150,153,...,207,207,207,207,206,206,206,204,203,202
1,6,155,157,156,156,156,157,156,158,158,...,69,149,128,87,94,163,175,103,135,149
2,2,187,188,188,187,187,186,187,188,187,...,202,201,200,199,198,199,198,195,194,195
3,2,211,211,212,212,211,210,211,210,210,...,235,234,233,231,230,226,225,222,229,163
4,12,164,167,170,172,176,179,180,184,185,...,92,105,105,108,133,163,157,163,164,179


In [ ]:
train_set=MyDataset(train_df)
valid_set=MyDataset(test_df)

In [ ]:
len(train_set)

27455

In [ ]:
train_set[0]

(tensor([[[0.4196, 0.4627, 0.4980, 0.5255, 0.5451, 0.5608, 0.5725, 0.5882,
           0.6000, 0.6118, 0.6196, 0.6275, 0.6392, 0.6471, 0.6235, 0.6510,
           0.6588, 0.6667, 0.6667, 0.6706, 0.6706, 0.6706, 0.6745, 0.6706,
           0.6706, 0.6667, 0.6667, 0.6627],
          [0.4353, 0.4745, 0.5059, 0.5294, 0.5529, 0.5647, 0.5804, 0.5922,
           0.6039, 0.6157, 0.6275, 0.6392, 0.6431, 0.6667, 0.4667, 0.5961,
           0.6706, 0.6706, 0.6667, 0.6706, 0.6745, 0.6745, 0.6745, 0.6745,
           0.6745, 0.6706, 0.6706, 0.6667],
          [0.4431, 0.4824, 0.5137, 0.5373, 0.5569, 0.5686, 0.5882, 0.5961,
           0.6078, 0.6196, 0.6314, 0.6392, 0.6431, 0.6745, 0.4118, 0.5569,
           0.6667, 0.6706, 0.6706, 0.6706, 0.6745, 0.6745, 0.6784, 0.6784,
           0.6745, 0.6706, 0.6706, 0.6706],
          [0.4549, 0.4902, 0.5216, 0.5451, 0.5608, 0.5725, 0.5922, 0.6000,
           0.6118, 0.6235, 0.6353, 0.6392, 0.6549, 0.6549, 0.3725, 0.5647,
           0.6706, 0.6745, 0.6745, 0.6745, 

In [ ]:
train_loader=DataLoader(train_set,batch_size=64,shuffle=True,num_workers=4)
test_loader=DataLoader(valid_set,batch_size=64,shuffle=True,num_workers=4)

/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


In [ ]:
model=nn.Sequential(
    # This is first convo block
    #no need to flatten here. 28*28 images are fed into CNN
    #in_channels=1 bcoz its a gray scale image
    #out_channels=25 no. of kernels

    nn.Conv2d(in_channels=1,out_channels=25,kernel_size=3,stride=1,padding=1), # shape of single image=25*28*28
    nn.BatchNorm2d(25), #to normalize values of all the kernels
    nn.ReLU(),
    nn.MaxPool2d(kernel_size=2,stride=2), # image is made half 25*14*14

    #2nd Convo block
    nn.Conv2d(in_channels=25,out_channels=50,kernel_size=3,stride=1,padding=1), # shape of single image=50*14*14
    nn.BatchNorm2d(50), #to normalize values of all the kernels
    nn.ReLU(),
    nn.MaxPool2d(kernel_size=2,stride=2), # image is made half 50*7*7

    #3rd Convo block
    nn.Conv2d(in_channels=50,out_channels=75,kernel_size=3,stride=1,padding=1), # shape of single image=75*7*7
    nn.BatchNorm2d(75), #to normalize values of all the kernels
    nn.ReLU(),
    nn.MaxPool2d(kernel_size=2,stride=2), # image is made half 75*3*3

    nn.Flatten(),
    nn.Linear(75*3*3,512),
    nn.ReLU(),
    nn.Linear(512,24) #output layer.


)

In [ ]:
#Checking for GPU
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu') #the device to use for computations (GPU if available, otherwise CPU).
device
model=torch.compile(model).to(device) # Compiles the model using torch.compile for potential performance gains (e.g. using TorchDynamo or AOTAutograd) and moves it to the specified device (GPU if available, otherwise CPU).
#it just initializes weights and biases

In [ ]:
#here we dont have model.compile, model.fit. Here we need to do custom training.we have to write it ourselves
optimizer=torch.optim.Adam(model.parameters(),lr=3e-4)
loss_fn=nn.CrossEntropyLoss()   #by default its sparse categorical

In [ ]:
#No builtin function for accuracy over here. we have to write function for batch accuracy on our own. here accuracy is computed in batches
def batch_accuracy(output, y, N):
    pred = torch.argmax(output,dim=1)
    correct = (pred==y).sum().item()    # `.item()` extracts the value from the resulting tensor as a Python number
    return correct / N

In [ ]:
epochs=10
def training():
  for i in range(epochs):
    loss=0
    accuracy=0
    model.train()
    for indx,(image,label) in enumerate(train_loader):
      image=image.to(device)  # send data to GPU as we did for model
      label=label.to(device)
      output=model(image)
     #forward propagation till here

      batch_loss=loss_fn(output,label)
      #backward propagation starts here

      optimizer.zero_grad() #initially batch gradients are set to zero
#Its primary function is to reset the gradients of the model's parameters (weights and biases) to zero before starting the backpropagation for the next batch of data.

      batch_loss.backward()  #It initiates the backpropagation process, which is the core of how neural networks learn.
# It calculates the gradients of the loss function with respect to all the model's parameters (weights and biases) that have requires_grad=True (meaning they are trainable).

      optimizer.step()  # update parameters
      #It's the step where the optimizer actually updates the model's parameters (weights and biases) based on the gradients that were calculated during backpropagation (batch_loss.backward()).

      loss+=batch_loss.item() #accumulate the loss values across multiple batches during the training process.
      accuracy+=batch_accuracy(output,label,label.shape[0])  ##label is a one-dimensional array. This gives number of samples in a batch, just like N
    print(f'Training Epoch: {i+1}, Accuracy: {accuracy/len(train_loader)}, Loss: {loss/len(train_loader)}')

    #len(train_loader)=number of batches